In [13]:
from qonnx.core.modelwrapper import ModelWrapper
import onnxruntime as ort
from onnx import helper
from onnxruntime_extensions import get_library_path
import numpy as np

In [52]:
def revert_quantAvgPool(model):
    nodes = [n for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    attrs = [n.attribute for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    for node,attr in zip(nodes,attrs):
        for a in attr:
            if a.name == "stride":
                s = a.i
            elif a.name == "kernel":
                k = a.i
        update = helper.make_node(
            "AveragePool",
            inputs=[node.input[0]],
            outputs=[node.output[0]],
            kernel_shape=[k,k],
            strides=[s,s],
        )

        model.graph.node.remove(node)
        model.graph.node.append(update)
        
def set_multithreshold_default(model,save_model):

    new_attr = [helper.make_attribute("out_scale", 1.0),
                helper.make_attribute("out_bias", 0.0),
                helper.make_attribute("data_layout","NCHW")]

    for n in model.graph.node:
        if n.op_type == "MultiThreshold":
            out_scale,bias,datalayout = False,False,False
            for na in n.attribute:
                if na.name == "out_scale": out_scale = True
                if na.name == "out_bias": bias = True
                if na.name == "data_layout": datlayout = True
            if not out_scale: n.attribute.append(new_attr[0])
            if not bias: n.attribute.append(new_attr[1])
            if not datalayout: n.attribute.append(new_attr[2])

            n.domain = "ai.onnx.contrib"
    model.save(mfile_up)

def ort_implementation(x,model):
    so = ort.SessionOptions()
    so.register_custom_ops_library(get_library_path())

    sess = ort.InferenceSession(mfile_up, so)
    print(sess.get_inputs()[0].shape)
    print(x.shape)
    expect_inp_shape = sess.get_inputs()[0].shape
    x = np.reshape(x,tuple(expect_inp_shape))
    inp_dict = {}
    for inp in sess.get_inputs():
        inp_dict[inp.name] = x
    res = sess.run([],inp_dict)
    return res   


model_dir = "../../../../../../models/estimate/Mul/Mul_0_out0/"
mfile = model_dir+"partition_1.onnx"
mfile_up = mfile.replace(".onnx","_ort.onnx")

model = ModelWrapper(mfile)
revert_quantAvgPool(model)
set_multithreshold_default(model,mfile_up)
ort_out = ort_implementation(np.load("output.npy"),model)
# np.save(model_dir +"ort_out.npy",ort_out)
ort_out

[1, 32, 111, 111]
(1, 111, 111, 32)


[array([[846, 623, 499, 451, 556]], dtype=int64)]

In [10]:
import numpy as np
import os
from qonnx.core.datatype import DataType
from driver_base import FINNExampleOverlay

input_file = "../../../../../../models/estimate/Mul/Mul_0_out0/input.npy"
x = np.load(input_file)
x = x.reshape([1,224,224,3]).astype(np.int8)
print(x.dtype)
np.save("../../../../../../models/estimate/Mul/Mul_0_out0/input_hw.npy",x)
# deploy_dir = 'build-KV260/fit/Mul/Mul_0_out0/deploy'
! python driver.py --bitfile="../bitfile/finn-accel.bit" --inputfile="../../../../../../models/estimate/Mul/Mul_0_out0/input_hw.npy"

int8


In [42]:
out_hw = np.load("output.npy")
print("HW:", out_hw.shape)
ort_out = np.load(model_dir+"ort_out.npy")
print("ORT:", ort_out.shape)

ort_out = np.reshape(ort_out,out_hw.shape,order='a')
print("ORT:", out_hw.shape)
np.isclose(out_hw,ort_out)

HW: (1, 111, 111, 32)
ORT: (1, 1, 32, 111, 111)
ORT: (1, 111, 111, 32)


array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T